In [11]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import torch
import wandb
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

REBUILD_DATA = False
KERNEL_SIZE = 5
LEARNING_RATE = 0.0005

In [12]:
device = torch.device("cuda:0")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU run.")
else:
    device = torch.device("cpu")
    print("CPU run.")

wandb.config = {
}
wandb.init(project="cats-and-dogs")

GPU run.


In [13]:
class DogsVsCats():
    IMG_SIZE = 50
    CATS = "kagglecatsanddogs_5340\PetImages\Cat"
    DOGS = "kagglecatsanddogs_5340\PetImages\Dog"
    LABELS = {CATS: 0, DOGS: 1}
    
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        
        # iterate over directories
        for label in self.LABELS:
            print(label)
            
            # iterate over files
            for f in  tqdm(os.listdir(label)):
                try: 
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])

                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    #print(str(e))
                    pass
            
        # shuffle data in place
        np.random.shuffle(self.training_data)
        np.save("catsdogs_training_data.npy", self.training_data)
        print("Cats:", self.catcount)
        print("Dogs:", self.dogcount)
        
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=KERNEL_SIZE, device=device)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=KERNEL_SIZE, device=device)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=KERNEL_SIZE, device=device)
        
        # cheat the size of input to linear layer
        x = torch.randn(DogsVsCats.IMG_SIZE, DogsVsCats.IMG_SIZE).to(device).view(-1, 1, DogsVsCats.IMG_SIZE, DogsVsCats.IMG_SIZE)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(in_features=self._to_linear, out_features=512, device=device)
        self.fc2 = nn.Linear(in_features=512, out_features=2, device=device)
    
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
        
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(input=self.fc1(x))
        x = self.fc2(x)
        return F.softmax(input=x, dim=1)
        
dogsvscats = DogsVsCats()
if REBUILD_DATA:
    dogsvscats.make_training_data()
net = Net().to(device)
wandb.watch(net)

optimizer = optim.Adam(params=net.parameters(), lr=LEARNING_RATE)
loss_function = nn.MSELoss()

print(net)

kagglecatsanddogs_5340\PetImages\Cat


100%|████████████████████████████████████████████████████████████████████████████████████████| 12501/12501 [00:11<00:00, 1083.34it/s]


kagglecatsanddogs_5340\PetImages\Dog


100%|████████████████████████████████████████████████████████████████████████████████████████| 12501/12501 [00:12<00:00, 1031.04it/s]
C:\Python39\lib\site-packages\numpy\lib\npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Cats: 12476
Dogs: 12470
Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [14]:
# set up data

training_data = np.load("catsdogs_training_data.npy", allow_pickle=True)

X = torch.Tensor([i[0] for i in training_data]).to(device).view(-1, dogsvscats.IMG_SIZE, dogsvscats.IMG_SIZE)
X = X/255.0 # scaling the image changing RGB scale to a decimal
y = torch.Tensor([i[1] for i in training_data]).to(device)

VAL_PCT = 0.1 # test against a percent of the data set
val_size = int(len(X)*VAL_PCT) # size of validation set
print(f"Expected size of test set, at {VAL_PCT*100}% of total set:", val_size)
# split training set to create test and train sets
train_X = X[:-val_size].to(device)
train_y = y[:-val_size].to(device)
print(f"Size of train set:", len(train_X))

test_X = X[-val_size:].to(device)
test_y = y[-val_size:].to(device)
print(f"Size of test set:", len(test_X))

Expected size of test set, at 10.0% of total set: 2494
Size of train set: 22452
Size of test set: 2494


In [15]:
def fwd_pass(X, y, train=False):
    if train:
        net.zero_grad()
    outputs = net(X).to(device)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    accuracy = matches.count(True)/len(matches)
    loss = loss_function(outputs, y).to(device)
    
    if train:
        loss.backward()
        optimizer.step()
    return accuracy, loss

def test(size=32):
    X, y = test_X[:size], test_y[:size]
    accuracy, loss = fwd_pass(X.view(-1, 1, dogsvscats.IMG_SIZE, dogsvscats.IMG_SIZE), y)
    return accuracy, loss

In [16]:
def train():
    BATCH_SIZE = 100
    EPOCHS = 30
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, dogsvscats.IMG_SIZE, dogsvscats.IMG_SIZE).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)
            
            accuracy, loss = fwd_pass(batch_X, batch_y, train=True)
            if i % 9 == 0:
                eval_accuracy, eval_loss = test(size=BATCH_SIZE)
                wandb.log({'evaluation loss': round(float(eval_loss),4), 'eval accuracy': round(float(eval_accuracy),4), 'loss': round(float(loss),4), 'accuracy': round(float(accuracy),4), 'epoch': epoch, 'batch': i/BATCH_SIZE})

In [17]:
train()

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 225/225 [00:03<00:00, 62.33it/s]


In [18]:
wandb.finish()

accuracy,▁▁▂▃▄▅▄▅▅▆▅▆▅▅▆▇▆▆▆▆▇▅█▇▇▇▇█▆▆██▇▇▇██▇██
batch,▃█▅▅▂▇▇▄▁▁▆▃▃▇▇▅▂▂▆▄▃█▅▅▂▇▇▄▄▁▆▆▃█▇▅▂▂▇▆
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
eval accuracy,▂▁▃▅▆▆▆▆▆▇▆▆▇█▇▇▆▆▇▆▇▇▇▆▇▆▆▆▆▆▅▅▆▆▆▆▆▅▆▆
evaluation loss,▇█▇▅▄▅▃▄▃▃▄▂▃▁▃▄▅▄▄▅▃▄▄▆▄▅▇▅▆▇▇█▇█▇▇▇██▆
loss,██▇▆▆▅▆▅▅▄▅▄▅▄▄▃▄▄▄▄▃▄▂▂▃▂▂▂▃▄▂▁▂▂▂▁▁▂▂▂
accuracy,0.93
batch,216.0
epoch,29
eval accuracy,0.74
evaluation loss,0.2202
